In [6]:
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.keys import Keys

import pandas as pd
import openpyxl as xl
import random

In [7]:
FILE_NAME='Z条件_データ1_定点'
SHEET_NAME='Z打鍵'

#データ読み込み
df=pd.read_excel(FILE_NAME+'.xlsm',sheet_name=SHEET_NAME)
data=df.loc[30,:].to_dict()

/Users/s87gawa/miniforge3/envs/py38/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [8]:
options = webdriver.ChromeOptions()
#options.add_argument('--headless') #ブラウザ表示なし
options.add_argument('--incognito') #シークレットモード 
browser = webdriver.Chrome(options=options)

#見積もりページを開く
url= "https://www.zurich.co.jp/auto/common/ncdAssessmentPage.html"
browser.get(url)


In [9]:
data['NF2'] = '20 等級'
#data['NF2'] = '6S'


In [10]:
if 'S' in data['NF2']:
    new = 'New'
    browser.find_element(By.CSS_SELECTOR, '#page1_question1_answer2 > div > span.txt > p.btnpara').click()#新規
    sleep(1)
    browser.find_element(By.CSS_SELECTOR, '#btnQ1_HasNoBusiness > a > div > dl > dt').click()#過去契約がない

    #セカンドカー
    if data['NF2']=='7S':
        browser.find_element(By.CSS_SELECTOR, '#btnQ2_HasAnotherCar > a > div > dl > dd').click()
        browser.find_element(By.CSS_SELECTOR, '#btnQ3_ApplySecondCar > a > div > dl > dt').click()
        browser.find_element(By.CSS_SELECTOR, '#part_newSecondCarBtnBlock > p > a').click()

    else:
        browser.find_element(By.CSS_SELECTOR, '#btnQ2_NoAnotherCar > a > div > dl > dt').click()
        browser.find_element(By.CSS_SELECTOR, '#part_newBusinessBtnBlock > p > a > span').click()
else:
    new = ''
    browser.find_element(By.CSS_SELECTOR, '#page1_question1_answer1 > div > span.txt > p.btntxt > img').click()

if 'S' in data['NF2']:
    pass
else:
    #一年契約
    sleep(3)
    browser.find_element(By.CSS_SELECTOR, '#riskFactorForm\:selectYearType > tbody > tr > td.firstChild > label').click()   
    sleep(1)

#始期日
Select(browser.find_element(By.CSS_SELECTOR, '#riskFactor'+ new + 'Form\:commencementDateEraYearField')).select_by_visible_text(data['西暦2']) 
sleep(1)
Select(browser.find_element(By.CSS_SELECTOR, '#riskFactor'+ new + 'Form\:commencementDateMonthField')).select_by_visible_text(str(data['月2']))
sleep(1)
Select(browser.find_element(By.CSS_SELECTOR, '#riskFactor'+ new + 'Form\:commencementDateDayField')).select_by_visible_text(str(data['日2'])) 
sleep(1)

if 'S' in data['NF2']:
    pass
else:
    #加入中の保険会社
    Select(browser.find_element(By.CSS_SELECTOR,'#riskFactorForm\:insuranceCompanyCodeField')).select_by_index(1)
    sleep(3)
    #現在の等級
    Select(browser.find_element(By.CSS_SELECTOR,'#riskFactorForm\:currentNoClaimDiscountField')).select_by_visible_text(data['NF2'])
    sleep(3)
    #事故件数
    browser.find_element(By.CSS_SELECTOR, '#riskFactorForm\:noPreviousClaimsField > tbody > tr > td.firstChild > label').click()
    sleep(1)

    #事故有係数
    browser.find_element(By.CSS_SELECTOR, '#riskFactorForm\:prevYrAccidentPeriodCountField > tbody > tr > td.firstChild > label').click()
    sleep(1)

    #車両保険あり
    browser.find_element(By.CSS_SELECTOR, '#riskFactorForm\:ownDamageCoverageField > tbody > tr > td.firstChild > label').click()
    #sleep(1)                                         
    #運限
    if data['運限修正'] == '本配':
        browser.find_element(By.CSS_SELECTOR, '#riskFactorForm\:driverLimitationDiscountField > tbody > tr > td.nthChild2n.nthChild2 > label').click()
    elif data['運限修正'] == '家族':
        browser.find_element(By.CSS_SELECTOR, '#riskFactorForm\:driverLimitationDiscountField > tbody > tr > td.firstChild > label').click()
    else:
        browser.find_element(By.CSS_SELECTOR, '#riskFactorForm\:driverLimitationDiscountField > tbody > tr > td:nth-child(4) > label').click()
    sleep(1)

    #年齢
    Select(browser.find_element(By.CSS_SELECTOR,'#riskFactorForm\:ageLimitedDiscountField')).select_by_visible_text(data['年齢限定修正2'])
    #sleep(1)

#初度登録
Select(browser.find_element(By.CSS_SELECTOR, '#riskFactor'+ new + 'Form\:carRegistrationDateEraYearField')).select_by_visible_text(data['初度年2']) 
sleep(1)
Select(browser.find_element(By.CSS_SELECTOR, '#riskFactor'+ new + 'Form\:carRegistrationDateMonthField')).select_by_visible_text(str(data['初度月2']))
sleep(1)

#型式
browser.find_element(By.CSS_SELECTOR, '#riskFactor'+ new + 'Form\:carTypeField_input').send_keys(data['型式2'])
browser.find_element(By.CSS_SELECTOR, '#riskFactor'+ new + 'Form\:autoCompleteButton').click()
sleep(2)
Select(browser.find_element(By.CSS_SELECTOR, '#riskFactor'+ new + 'Form\:dummyCarTypeListField0')).select_by_index(1)
sleep(1)

#記名被保険者は契約者の配偶者男性
browser.find_element(By.CSS_SELECTOR, '#riskFactor'+ new + 'Form\:mainDriverRelationField > tbody > tr > td.nthChild3.nthChild3n > label').click()

#生年月日
Select(browser.find_element(By.CSS_SELECTOR, '#riskFactor'+ new + 'Form\:insuredDOBEraYearField')).select_by_visible_text(data['生年2']) 
sleep(1)
Select(browser.find_element(By.CSS_SELECTOR, '#riskFactor'+ new + 'Form\:insuredDOBMonthField')).select_by_visible_text(str(data['生まれ月2']))
sleep(1)
Select(browser.find_element(By.CSS_SELECTOR, '#riskFactor'+ new + 'Form\:insuredDOBDayField')).select_by_visible_text(str(data['生まれ日2'])) 
sleep(1)
Select(browser.find_element(By.CSS_SELECTOR, '#riskFactor'+ new + 'Form\:mainDriverResidencialAreaField')).select_by_visible_text(str(data['地域2'])) 
sleep(1)

#免許の色
if data['免許2'] == 'ゴールド':
    browser.find_element(By.CSS_SELECTOR, '#riskFactor'+ new + 'Form\:driversLicenseColorAutoField > tbody > tr > td.firstChild > label').click()
else:
    browser.find_element(By.CSS_SELECTOR, '#riskFactor'+ new + 'Form\:driversLicenseColorAutoField > tbody > tr > td.nthChild2 > label').click()

sleep(1)
#目的
if data['使用目的2'] == '日常':
    browser.find_element(By.CSS_SELECTOR, '#riskFactor'+ new + 'Form\:vehicleUsageField > tbody > tr > td.firstChild > label').click()
else:
    browser.find_element(By.CSS_SELECTOR, '#riskFactor'+ new + 'Form\:vehicleUsageField > tbody > tr > td.nthChild3.nthChild3n > label').click()#通勤25km
sleep(1)

#目的
if data['走行距離2'] == 1:
    browser.find_element(By.CSS_SELECTOR, '#riskFactor'+ new + 'Form\:anualMilleageField > tbody > tr > td.firstChild > label').click()
elif data['走行距離2'] == 2:
    browser.find_element(By.CSS_SELECTOR, '#riskFactor'+ new + 'Form\:anualMilleageField > tbody > tr > td.nthChild2 > label').click()
elif data['走行距離2'] == 3:
    browser.find_element(By.CSS_SELECTOR, '#riskFactor'+ new + 'Form\:anualMilleageField > tbody > tr > td.nthChild3 > label').click()
elif data['走行距離2'] == 4:
    browser.find_element(By.CSS_SELECTOR, '#riskFactor'+ new + 'Form\:anualMilleageField > tbody > tr > td:nth-child(4) > label').click()
elif data['走行距離2'] == 5:
    browser.find_element(By.CSS_SELECTOR, '#riskFactor'+ new + 'Form\:anualMilleageField > tbody > tr > td:nth-child(5) > label').click()
sleep(1)

if 'S' in data['NF2']:
    #運限
    if data['運限修正'] == '本配':
        browser.find_element(By.CSS_SELECTOR, '#riskFactorNewForm\:driverLimitationDiscountField > tbody > tr > td.firstChild > label').click()
    elif data['運限修正'] == '家族':
        browser.find_element(By.CSS_SELECTOR, '#riskFactorNewForm\:driverLimitationDiscountField > tbody > tr > td.nthChild2n.nthChild2 > label').click()
    else:
        browser.find_element(By.CSS_SELECTOR, '#riskFactorNewForm\:driverLimitationDiscountField > tbody > tr > td:nth-child(3) > label').click()
    sleep(1)

    #年齢
    if data['年齢限定修正2'] == '全年齢補償':
        browser.find_element(By.CSS_SELECTOR, '#riskFactorNewForm\:ageLimitedDiscountField > tbody > tr > td.firstChild > label').click()
    elif data['年齢限定修正2'] == '21歳以上補償':
        browser.find_element(By.CSS_SELECTOR, '#riskFactorNewForm\:ageLimitedDiscountField > tbody > tr > td.nthChild2n.nthChild2 > label').click()
    elif data['年齢限定修正2'] == '26歳以上補償':
        browser.find_element(By.CSS_SELECTOR, '#riskFactorNewForm\:ageLimitedDiscountField > tbody > tr > td:nth-child(3) > label').click()
    else:
        browser.find_element(By.CSS_SELECTOR, '#riskFactorNewForm\:ageLimitedDiscountField > tbody > tr > td:nth-child(4) > label').click()
    sleep(1)

    #セカンドカー有りの場合、他の保険会社
    if len(browser.find_elements(By.CSS_SELECTOR,'#riskFactorNewForm\:insuranceCompanyCodeField'))>0:
        Select(browser.find_element(By.CSS_SELECTOR,'#riskFactorNewForm\:insuranceCompanyCodeField')).select_by_index(1)
else:
    pass

#何で知ったか
browser.find_element(By.CSS_SELECTOR, '#riskFactor'+ new + 'Form\:mediaField > tbody > tr > td.nthChild3.nthChild3n > label').click()
sleep(1)

#次へ
browser.find_element(By.CSS_SELECTOR, '#riskFactor'+ new + 'Form\:button > img').click()
sleep(1)

#はい
browser.find_element(By.CSS_SELECTOR, '#riskFactor'+ new + 'Form\:yes_button').click()


#お客さま情報

browser.find_element(By.CSS_SELECTOR, '#car-01-01').send_keys('日本')
browser.find_element(By.CSS_SELECTOR, '#car-01-02').send_keys('太郎')
browser.find_element(By.CSS_SELECTOR, '#car-02-01').send_keys('ニホン')
browser.find_element(By.CSS_SELECTOR, '#car-02-02').send_keys('タロウ')

#女性、既婚
browser.find_element(By.CSS_SELECTOR, '#birthBox > div > dl:nth-child(1) > dd > ul > li.btn03.genderF.noset').click()
browser.find_element(By.CSS_SELECTOR, '#birthBox > div > dl:nth-child(2) > dd > ul > li.btn03.married.noset').click()


#2/29に設定する
browser.find_element(By.CSS_SELECTOR, '#nen').send_keys('1980')
browser.find_element(By.CSS_SELECTOR, '#tsuki').send_keys('2')
browser.find_element(By.CSS_SELECTOR, '#hi').send_keys('29')

#住所は適当
browser.find_element(By.CSS_SELECTOR, '#zip').send_keys('1110042')
sleep(1)
browser.find_element(By.CSS_SELECTOR, '#addressBox > div > dl.clearfix.required > dd > ul > li:nth-child(1)').click()
browser.find_element(By.CSS_SELECTOR, '#add02').send_keys('１−１−１')


#適当なメルアド。重複すると弾かれる。
tel = random.randrange(10**11,10**12)
browser.find_element(By.CSS_SELECTOR, '#car-08-01').send_keys('0'+str(tel))
browser.find_element(By.CSS_SELECTOR, '#car-12-01').send_keys(str(tel) + '@zurich.co.jp')
browser.find_element(By.CSS_SELECTOR, '#car-12-03').send_keys('zurickpass')

sleep(1)
browser.find_element(By.CSS_SELECTOR, '#sendBlock').click()

##結果画面

sleep(1)
browser.find_element(By.CSS_SELECTOR, '#calculatePremiumForm\:quotePremium1_top').text

browser.find_element(By.CSS_SELECTOR, '#calculatePremiumForm\:planComment2').click()

Select(browser.find_element(By.CSS_SELECTOR,'#calculatePremiumForm\:ownDamageOption_plan2')).select_by_visible_text('なし')

#保険料再計算
try:
    browser.find_element(By.CSS_SELECTOR, '#calculatePremiumForm\:recalculateBtn2_top > span > img').click()
except:
    pass

sleep(2)

browser.find_element(By.CSS_SELECTOR, '#calculatePremiumForm\:quotePremium2_top').text

'15,510'